## DFT Course - 2022-23
## Beatriz Helena Cogollo-Olivo

## Tutorial 03 - Electronic structure in action

This example illustrates the use of pw.x as a starting point to calculate material properties such as band structure and the electronic density of states for silicon.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Initial calculations

The first step is to perform a SCF calculation to obtain the wavefunctions that will be used in subsequent calculations.

In [ ]:
#Let's download the pseudopotential
!wget http://pseudopotentials.quantum-espresso.org/upf_files/Si.pz-vbc.UPF

In [ ]:
%%writefile Si-SCF.in

&control
    calculation = 'scf',
    prefix = 'silicon',
    wf_collect = .true. ,
    outdir = './tmp/',
    pseudo_dir = './',
 /
 &system
    ibrav =  2,
    celldm(1) = 10.2076,
    nat =  2,
    ntyp = 1,
    ecutwfc = 40,
    ecutrho = 320,
    nbnd = 12,
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  6 6 6 1 1 1

In [ ]:
!pw.x < Si-SCF.in > Si-SCF.out

In [ ]:
!grep "highest occupied" Si-SCF.out

#### Question 1 
What is the value of the bandgap, if any?

### Bands calculation

Once we finished the initial calculations, we can proceed with the density of states (DOS) calculation.

In [ ]:
%%writefile Si-Pre-BANDS.in

&control
    calculation = 'bands',
    prefix = 'silicon',
    wf_collect = .true. ,
    outdir = './tmp/',
    pseudo_dir = './',
 /
 &system
    ibrav =  2,
    celldm(1) = 10.2076,
    nat =  2,
    ntyp = 1,
    ecutwfc = 40,
    ecutrho = 320,
    nbnd = 12,
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS {crystal_b}
5
  0.0000 0.5000 0.0000 20  !L
  0.0000 0.0000 0.0000 30  !G
  -0.500 0.0000 -0.500 10  !X
  -0.375 0.2500 -0.375 30  !U
  0.0000 0.0000 0.0000 20  !G

In [ ]:
!pw.x < Si-Pre-BANDS.in > Si-Pre-BANDS.out

In [ ]:
%%writefile Si-BANDS.in

&BANDS
  prefix = 'silicon'
  outdir = './tmp/'
  filband = 'si_bands.dat'
/

In [ ]:
!bands.x < Si-BANDS.in > Si-BANDS.out

The following file will define the parameters for the electronic bands:

In [ ]:
%%writefile Si-PLOTBAND.in

si_bands.dat
-6, 16                 !Y limits
si_bands.gnuplot
si_bands.ps
FERMI                  !Highest occupied value
4, 0                   !Delta E, tick for reference (Fermi) level

In [ ]:
!plotband.x < Si-PLOTBAND.in > Si-PLOTBAND.out

In [ ]:
!evince si_bands.ps

In [ ]:
!grep "high-symmetry point" Si-PLOTBAND.out

In [ ]:
data = np.loadtxt('si_bands.dat.gnu')

k = np.unique(data[:, 0])
bands = np.reshape(data[:, 1], (-1, len(k)))

for band in range(len(bands)):
    plt.plot(k, bands[band, :], linewidth=1, alpha=0.5, color='k')
plt.xlim(min(k), max(k))

# Fermi energy
plt.axhline(FERMI, linestyle=(0, (5, 5)), linewidth=0.75, color='k', alpha=0.5) #Set the reference (Fermi) leve
# High symmetry k-points (check bands_pp.out)
plt.axvline(HIGH-SYMMETRY-POINT, linewidth=0.75, color='k', alpha=0.5) #Repeat for each high-symmetry point reported
# text labels
#List the high-symmetry points reported in the output file
plt.xticks(ticks= [HIGH-SYMMETRY-POINT,...], \
           labels=['POINT', ...])                #List the names of the high-symmetry points    
plt.ylabel("Energy (eV)")
plt.text(2.3, 5.6, 'Fermi energy')
plt.show()

#### Question 2
Is the system a metal, a semiconductor, or an insulator? Why?

### DOS calculation

Before performing the DOS calculation, we need a NSCF calculation (after the SCF calculation). For the NSCF it is necesary a denser K-point grid. The card $occupations$ is included with the option $tetrahedra$, which is appropriate for DOS calculation. In addition, we need to specify $nosym = .TRUE.$ to avoid generation of additional k-points in low symmetry cases. Finally, $outdir$ and $prefix$ must be same as in the SCF step.

In [ ]:
%%writefile Si-NSCF.in

&control
    calculation = 'nscf',
    prefix = 'silicon',
    outdir = './tmp/',
    pseudo_dir = './',
 /
 &system
    ibrav =  2,
    celldm(1) = 10.2076,
    nat =  2,
    ntyp = 1,
    ecutwfc = 40,
    occupations = 'tetrahedra',
    nosym = .true.
 /
 &electrons
    mixing_beta = 0.6
 /
ATOMIC_SPECIES
 Si 28.086  Si.pz-vbc.UPF
ATOMIC_POSITIONS (alat)
 Si 0.0 0.0 0.0
 Si 0.25 0.25 0.25
K_POINTS (automatic)
  12 12 12 1 1 1

In [ ]:
!pw.x < Si-NSCF.in > Si-NSCF.out

In [ ]:
!grep "Fermi" Si-NSCF.out

#### Question 3
What is the value of the Fermi level?

In [ ]:
%%writefile Si-DOS.in

&DOS
  prefix='silicon',
  outdir='./tmp/',
  fildos='si_dos.dat',
  emin=-9.0,  !Energy lower limit
  emax=16.0   !Energy upper limit
/

In [ ]:
!dos.x < Si-DOS.in > Si-DOS.out

In [ ]:
# load data
energy, dos, idos = np.loadtxt('si_dos.dat', unpack=True)

# make plot
plt.plot(energy, dos, linewidth=0.75, color='red')
plt.yticks([])
plt.xlabel('Energy (eV)')
plt.ylabel('DOS')
plt.axvline(x=FERMI, linewidth=0.5, color='k', linestyle=(0, (8, 10)))  #Set the Fermi level
plt.xlim(LOWER, UPPER)  #Set the X limits
plt.ylim(0, )
plt.fill_between(energy, 0, dos, where=(energy < FERMI), facecolor='red', alpha=0.25)
plt.text(6, 1.5, 'Fermi energy', rotation=90)
plt.show()

#### Question 4
Is this graph consistent with the plotted bands?

### PDOS calculation

In addition to the standard DOS calculation, we can determine the orbital contribution from each individual atoms.

In [ ]:
%%writefile Si-PDOS.in

&PROJWFC
  prefix= 'silicon',
  outdir= './tmp/',
  filpdos= 'si_pdos.dat'
/

In [ ]:
!projwfc.x < Si-PDOS.in > Si-PDOS.out

In [ ]:
def data_loader(fname):
    fid = open(fname, "r")
    data = fid.readlines()
    fid.close()

    energy = []
    pdos = []

    for row in range(len(data)):
        data_row = data[row]
        if (data_row[0][0] != '#'):
            data_row = data_row[:-1].split('  ')
            energy.append(float(data_row[1]))
            pdos.append(float(data_row[3]))

    energy = np.asarray(energy)
    pdos = np.asarray(pdos)

    return energy, pdos

energy, pdos_s1 = data_loader('si_pdos.dat.pdos_atm#1(Si)_wfc#1(s)')
_, pdos_s2 = data_loader('si_pdos.dat.pdos_atm#2(Si)_wfc#1(s)')
_, pdos_p1 = data_loader('si_pdos.dat.pdos_atm#1(Si)_wfc#2(p)')
_, pdos_p2 = data_loader('si_pdos.dat.pdos_atm#2(Si)_wfc#2(p)')
_, pdos_tot = data_loader('si_pdos.dat.pdos_tot')

# make plots
plt.figure(figsize = (8, 4))
plt.plot(energy, pdos_s1+pdos_s2, linewidth=0.75, color='#006699', label='s-orbital')
plt.plot(energy, pdos_p1+pdos_p2, linewidth=0.75, color='r', label='p-orbital')
plt.plot(energy, pdos_tot, linewidth=0.75, color='k', label='total')
plt.yticks([])
plt.xlabel('Energy (eV)')
plt.ylabel('DOS')
plt.axvline(x= FERMI, linewidth=0.5, color='k', linestyle=(0, (8, 10))) #Set the Fermi level
plt.xlim(LOWER, UPPER)   #Set the X limits
plt.ylim(0, )
plt.fill_between(energy, 0, pdos_s1+pdos_s2, where=(energy < FERMI), facecolor='#006699', alpha=0.25) #Set the Fermi level
plt.fill_between(energy, 0, pdos_p1+pdos_p2, where=(energy < FERMI), facecolor='r', alpha=0.25) #Set the Fermi level
plt.fill_between(energy, 0, pdos_tot, where=(energy < FERMI), facecolor='k', alpha=0.25) #Set the Fermi level
plt.text(6, 1.3, 'Fermi energy', rotation=90)
plt.legend(frameon=False)
plt.show()

#### Question 5
What can you interpretate from the graph?